In [ ]:
# Copyright (c) 2020-2023, NVIDIA CORPORATION.  All rights reserved.
#
# NVIDIA CORPORATION and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA CORPORATION is strictly prohibited.

In [ ]:
import sys
import numpy as np
from isaacsim import SimulationApp

simulation_app = SimulationApp(launch_config={"headless": True})
import omni

simulation_app.update()
omni.usd.get_context().new_stage()
simulation_app.update()


In [ ]:
from isaacsim.sensors.camera import Camera
from isaacsim.core.api.objects import VisualCuboid, DynamicCuboid
from omni.kit.viewport.utility import get_active_viewport

viewport_api = get_active_viewport()
render_product_path = viewport_api.get_render_product_path()
camera = Camera(
    prim_path="/World/camera",
    position=np.array([0.0, 0.0, 25.0]),
    resolution=(1280, 720),
    render_product_path = render_product_path
)
simulation_app.update()
camera.initialize()
simulation_app.update()
camera.add_distance_to_image_plane_to_frame()
camera.add_bounding_box_2d_tight_to_frame()
camera.add_bounding_box_2d_loose_to_frame()
camera.add_instance_segmentation_to_frame()
camera.add_semantic_segmentation_to_frame()
camera.add_bounding_box_3d_to_frame()
simulation_app.update()


VisualCuboid(
    prim_path="/new_cube_1",
    name="visual_cube",
    position=np.array([0, 0, 0.5]),
    scale=np.array([1, 1, 1]),
    color=np.array([255, 255, 255]),
)


In [ ]:
omni.timeline.get_timeline_interface().play()

for _ in range(10):
    simulation_app.update()

rgb = camera.get_rgba()

print(rgb.size)
if rgb.size != 1280 * 720 * 4:
    raise ValueError(f"RGB buffer has size of {rgb.size} which is not {1280*720*4}")


In [ ]:
import omni.graph.core as og
import omni

from omni.syntheticdata import sensors

simulation_app.update()
omni.timeline.get_timeline_interface().play()
viewport_api = get_active_viewport()
import omni.syntheticdata._syntheticdata as sd

sensors.enable_sensors(viewport_api, [sd.SensorType.DistanceToImagePlane])
simulation_app.update()
graph = og.ObjectLookup.graph("/Render/PostProcess/SDGPipeline")
raw_node = og.ObjectLookup.node(
    "/Render/PostProcess/SDGPipeline/PostProcessDispatcher"
)
swh_attr = og.Controller.attribute("outputs:referenceTimeNumerator", raw_node)
first = og.DataView.get(swh_attr)
simulation_app.update()
second = og.DataView.get(swh_attr)

if first + 1 != second:
    raise ValueError(f"swh frame numbers {first}, {second} should be one apart")


In [ ]:
# Cleanup application
simulation_app.close()
